In [1]:
import warnings
warnings.filterwarnings("ignore")

import re
import os
import random
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import json

from DataLoader.DataLoader import DataLoader
from DataLoader.DataBasedAgent import DataBasedAgent
from DataLoader.DataRLAgent import DataRLAgent
import DeepRLAgent.VanillaInput.Train as Train
from PatternDetectionInCandleStick.Evaluation import Evaluation
import distinctipy

from importlib import reload

Train = reload(Train)
DeepRL = Train.Train
from utils_best_arm import add_train_portfo, add_test_portfo, plot_return, calc_return, plot_action_point, setup_logger
pd.options.display.max_colwidth = 100

device = "cpu"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CURRENT_PATH = os.getcwd()

In [2]:
def find_best_arm(_file, begin_date, end_date):
    # TODO: find best arm of all seeds
    rewards = []
    for seed in range(100):
        path = f"./Results/{_file}/{begin_date}~{end_date}/{seed}/train_log/{seed}.log"
        with open(path, "r", encoding="utf8") as f:
            content = f.read()
            reward = re.findall(f"final reward type: (.*?)_{seed}\n", content)[0]
            rewards.append(reward)

    best_reward = sorted(collections.Counter(rewards,).items(), key=lambda x: x[1], reverse=True)[0][0]
    return best_reward


_file = "AAPL"
begin_date, end_date = "2016-01-01", "2019-01-01"
find_best_arm(_file, begin_date, end_date)

'regularized_0.1'

In [3]:
def train(
    DATASET_NAME, 
    split_point='2018-01-01', 
    begin_date='2010-01-01', 
    end_date='2020-08-24', 
    initial_investment=1000,
    transaction_cost=0.0001,
    load_from_file=True,
    seed=42, 
    state_mode=1,
    n_episodes=5,
    lamb=0.0001,
    GAMMA=0.7, 
    n_step=5, 
    BATCH_SIZE=10, 
    ReplayMemorySize=20,
    TARGET_UPDATE=5,
    window_size=None, 
    train_portfolios={},
    test_portfolios={},
    arms=[],
    show_all = False,
    ratio_threshold=0.9,
):
    data_loader = DataLoader(DATASET_NAME, split_point=split_point, begin_date=begin_date, end_date=end_date, load_from_file=load_from_file)
    
    dataTrain_agent = DataRLAgent(data_loader.data_train, state_mode, 'action_encoder_decoder', device, GAMMA, n_step, BATCH_SIZE, window_size, transaction_cost)
    dataTest_agent = DataRLAgent(data_loader.data_test, state_mode, 'action_encoder_decoder', device, GAMMA, n_step, BATCH_SIZE, window_size, transaction_cost)
    
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

    agent = DeepRL(data_loader, dataTrain_agent, dataTest_agent, 
                DATASET_NAME,  state_mode, window_size, transaction_cost,
                BATCH_SIZE=BATCH_SIZE, GAMMA=GAMMA, ReplayMemorySize=ReplayMemorySize,
                TARGET_UPDATE=TARGET_UPDATE, n_step=n_step, arms=arms)
    arm = arms[0]
    agent_test = agent.test(initial_investment=initial_investment, test_type='test', arm=arm)
    test_portfolio = agent_test.get_daily_portfolio_value()
    test_portfolio = pd.Series(test_portfolio).pct_change(1).fillna(0).values.tolist() 
    model_name = f'DQN-stock:{DATASET_NAME}-reward:{arm["name"]}-{arm["lamb"]}-seed:{seed}'
    return data_loader, {"name": model_name, "portfo": test_portfolio}

In [25]:
random_seeds = 100

# TS的选择结果
with open(f"./ts-run-results/[{random_seeds}]ts100.json", "r", encoding="utf8") as f:
    content = json.loads(f.read())

# greedy的选择结果
with open(f"./ts-run-results/[{random_seeds}]ts-greedy100.json", "r", encoding="utf8") as f:
    content = json.loads(f.read())

from collections import Counter

results_dt = {}
for symbol, dates in content.items():
    dt = {}
    for date, _ in dates.items():
        reward = sorted(Counter(_).items(), key=lambda x: x[1], reverse=True)[0][0]
        dt[date] = reward
    results_dt[symbol] = dt

dt = {
    "profit-0": "FP5",
    "regularized-0.01": "FPR-0.01",
    "regularized-0.05": "FPR-0.05",
    "regularized-0.1": "FPR-0.1",
    "regularized-0.2": "FPR-0.2",
}

_results_dt = pd.DataFrame(results_dt)
_results_dt = _results_dt.T
# _results_dt = _results_dt.replace(dt)

results_dt = _results_dt.to_dict(orient='index')

# _results_dt.to_csv("./ts-run-results/ts-choose.csv")
# _results_dt.to_csv("./ts-run-results/greedy-choose.csv")

In [26]:
cols = _results_dt.columns 
for col in cols:
    _results_dt[col] = _results_dt[col].replace({
        "profit-0": "FP5",
        "regularized-0.01": "FPR-0.01",
        "regularized-0.05": "FPR-0.05",
    })

_results_dt

,2019-06-23:2020-06-22,2020-06-22:2021-06-22,2021-06-23:2022-06-23
AAPL,FPR-0.05,FP5,FPR-0.05
AMGN,FP5,FP5,FPR-0.01
AXP,FPR-0.01,FPR-0.01,FP5
BA,FPR-0.01,FP5,FP5
CAT,FPR-0.05,FP5,FPR-0.01
CRM,FPR-0.05,FP5,FP5
CSCO,regularized-0.1,FPR-0.01,FP5
CVX,FPR-0.01,FP5,FP5
DIS,FP5,FP5,FP5
GS,FP5,FP5,FP5


In [18]:
initial_investment = 1000

kwargs = {
    "load_from_file": True, 
    "transaction_cost": 0.0000,
    "initial_investment": initial_investment,
    "state_mode": 1,
    "GAMMA": 0.7, 
    "n_step": 5, 
    "BATCH_SIZE": 10, 
    "ReplayMemorySize": 20,
    "TARGET_UPDATE": 5,
    "window_size": None, 
    "lamb": 0.0,
}

_begin_date = '20{}-01-01'
_end_date = '20{}-01-01'
_split_point = '20{}-01-01' 

arms = [
    { "name": "profit", "lamb": 0},
    { "name": "regularized", "lamb": 0.01},
    { "name": "regularized", "lamb": 0.05},
    { "name": "regularized", "lamb": 0.1},
    { "name": "regularized", "lamb": 0.2},
    { "name": "sharpe", "lamb": 0.01 },
    { "name": "volatility", "lamb": 10 },
]

dates = [
    ("2019-06-23", "2020-06-22", "2016-01-01", "2019-01-01"),
    ("2020-06-22", "2021-06-22", "2017-01-01", "2020-01-01"),
    ("2021-06-23", "2022-06-23", "2018-01-01", "2021-01-01"),
]

random_seeds = 100
results = []
files = sorted(os.listdir("./Data/"))
portfolios_saved = {}
date_indexes = []
for _file in files[:1]:
    _file = "JPM"
    dt = results_dt[_file]
    print(_file)
    portfolios_saved[_file] = {}
    results2 = []
    tmp_result = []
    for seed in tqdm(range(random_seeds)):
        
        ls = []
        bhs = []
    
        train_portfolios = {}
        test_portfolios = {}


        for idx, (date, reward) in enumerate(dt.items()):
            
            model_begin_date = dates[idx][2]
            model_end_date = dates[idx][3]
            
            split_point = date.split(":")[0]
            end_date = date.split(":")[1]

            path = f"./Results/{_file}/{model_begin_date}~{model_end_date}/{seed}/train"
            reward = reward.replace("-", "_")
            reward = "regularized_0.01"

            model_path = f"{path}/model_{reward}_{seed}.pkl"
            arm = {
                "name": reward.split("_")[0],
                "lamb": reward.split("_")[1],
                "model_dir": model_path,
            }
            kwargs.update({
                "begin_date": model_begin_date, 
                "end_date": end_date, 
                "split_point": split_point, 
                "DATASET_NAME": _file,
                "seed": seed,
                "n_episodes": 140,
                # NOTE 这个arms没有用的
                "arms": [arm],
                "show_all": True,
                "ratio_threshold": 3,
                "train_portfolios": train_portfolios,
                "test_portfolios": test_portfolios,
            })

            data_loader, model = train(**kwargs)
            ls.extend(model["portfo"])
            bh = data_loader.data_test_with_date["close"]
            bhs.append(bh)
        
        add_test_portfo(test_portfolios, seed, ls)
        if seed == 0: 
            bhs = pd.concat(bhs, axis=0)
            bh_percentage = bhs.pct_change(1).fillna(0).values
            add_test_portfo(test_portfolios, 'B&H', bh_percentage)
            # tmp_result.append(bh_percentage.tolist())
        indexes = calc_return(bh_percentage, test_portfolios)
        results2.append(indexes)
        tmp_result.append(ls) 

    portfolios_saved[_file] = tmp_result
    
    path = f"./Results/{_file}/exp3_concat"
    if not os.path.exists(path):
        os.mkdir(path)
    
    save_path = f"{path}/ts.csv"
    # save_path = f"{path}/greedy.csv"

    portfolios_saved[_file].insert(0, bh_percentage.tolist())
    _df_ = pd.DataFrame(portfolios_saved[_file]).T
    _df_[0] = _df_[0].shift(1)
    _df_ = _df_.fillna(0)
    _df_.to_csv(save_path, index=False)
 
    results2_df = pd.concat(results2, axis=1)
    results2_bh = results2_df["B&H"]
    del results2_df["B&H"]
    final = pd.concat([
        results2_bh,
        results2_df.median(axis=1)
    ], axis=1)
    final.columns = [f"{_file}-B&H", f"{_file}-concat"]
    results.append(final)


JPM


100%|██████████| 100/100 [00:57<00:00,  1.73it/s]


In [8]:
symbols = list(portfolios_saved.keys())[:]
ls = []
for symbol in symbols:
    tmp = pd.DataFrame(portfolios_saved[symbol]).T.iloc[:, 1:]
    tmp = ((1 + tmp).cumprod() - 1)
    cols = tmp.columns
    for col in cols:
        if tmp[col].isna().sum() > 0:
            tmp[col] = tmp[col].shift(1)
            tmp[col] = tmp[col].fillna(0)
    tmp = tmp.median(axis=1)
    ls.append(tmp)


results_cumreturn = pd.concat(ls, axis=1)
dates = []
_ = list(map(lambda bh: dates.extend(list(bh.index)), bhs))
total_df = pd.concat([
    pd.DataFrame(dates), 
    results_cumreturn
], axis=1)
total_df["date"] = total_df.iloc[:, 0]
total_df["date"] = total_df["date"].shift(3)
total_df = total_df.dropna()
total_df = total_df.set_index("date")
total_df = total_df.iloc[:, 1:]
total_df.columns = [f"{symbol}-MSR-TSE" for symbol in symbols]
# total_df.to_csv("./ts-run-results/[exp3]ts-returns.csv", )
total_df.to_csv("./ts-run-results/[exp3]ts-cumreturns.csv",)

# total_df.columns = [f"{symbol}-MSR-GME" for symbol in symbols]
# total_df.to_csv("./ts-run-results/[exp3]greedy-returns.csv", )
# total_df.to_csv("./ts-run-results/[exp3]greedy-cumreturns.csv",)
total_df

,JPM-MSR-TSE
date,
2019-06-24,0.008352
2019-06-25,0.023756
2019-06-26,0.046214
2019-06-27,0.054937
2019-06-28,0.056050
...,...
2022-06-13,1.514173
2022-06-14,1.504921
2022-06-15,1.500947


In [ ]:
single